In [ ]:
!pip install pyomo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install gurobipy

In [ ]:
import pandas as pd
import gurobipy
from pyomo.environ import *

In [ ]:
file_path = '/content/Housing Problem - parameter values.xlsx'

apartments = pd.read_excel(file_path, sheet_name = 'number_of_apartments')
profit_height_69 = pd.read_excel(file_path, sheet_name = 'profit_height_69')
profit_height_120 = pd.read_excel(file_path, sheet_name = 'profit_height_120')
profit_height_168 = pd.read_excel(file_path, sheet_name = 'profit_height_168')

In [ ]:
profits = {
    23: profit_height_69,
    40: profit_height_120,
    56: profit_height_168
}



---



---



---



---



In [ ]:
def create_model(floors, profit_df, apartments_df):
    model = ConcreteModel()

    # Sets
    sectors = ['social', 'middle', 'free']
    areas = [36, 42, 48, 52, 60, 68, 70, 71, 96, 131]
    owners = ['corporation', 'investor', 'private']
    configurations = ['aa', 'ab', 'ac', 'bb', 'bc', 'cc', 'cd', 'ce', 'dd', 'de', 'ee']

    # Parameters
    model.K = floors
    model.R = apartments_df.set_index(['area', 'configuration']).to_dict()['no_of_apartments']
    model.O = profit_df.set_index(['sector', 'area', 'owner']).to_dict()['profit']

    # Minimal floor area for sector and owner (α_ih)
    alpha = {
        ('social', 'corporation'): 40,
        ('middle', 'corporation'): 50,
        ('free', 'investor'): 60
    }

    # Minimum percentage of apartments for each sector
    beta = {
        'social': 0.4,
        'middle': 0.4,
        'free': 0
    }

    # Minimum average floor area for each sector
    delta = {
        'social': 40,
        'middle': 50,
        'free': 0
    }

    # Minimum percentage of apartments for each owner
    gamma = {
        'corporation': 0,
        'investor': 0.7,
        'private': 0
    }

    # Variables
    model.X = Var(configurations, domain=NonNegativeIntegers)  # Number of floors with configuration v
    model.Y = Var(sectors, areas, owners, domain=NonNegativeIntegers)  # Number of apartments in sector i with floor area j and owner h
    model.W = Var(configurations, owners, domain=NonNegativeIntegers)  # Number of floors of type v with owner h
    model.Z = Var(areas, owners, domain=NonNegativeIntegers)  # Number of apartments with floor area j and owner h

    # Objective: Maximize profit
    model.obj = Objective(expr=sum(model.O[(i, j, h)] * model.Y[i, j, h] for i in sectors for j in areas for h in owners), sense=maximize)

    # Constraints
    # 1. Total number of floors
    model.floor_constraint = Constraint(expr=sum(model.X[v] for v in configurations) == model.K)

    # 2. Total apartments consistency
    def total_apartments_rule(model, j):
        return sum(model.R[(j, v)] * model.X[v] for v in configurations) == sum(model.Y[i, j, h] for i in sectors for h in owners)
    model.total_apartments = Constraint(areas, rule=total_apartments_rule)

    # 3. Owner floor configuration
    def owner_floor_configuration_rule(model, v):
        return sum(model.W[v, h] for h in owners) == model.X[v]
    model.owner_floor_configuration = Constraint(configurations, rule=owner_floor_configuration_rule)

    # 4. No floors with multiple owners
    def no_multiple_owners_rule(model, j, h):
        return model.Z[j, h] == sum(model.W[v, h] * model.R[(j, v)] for v in configurations)
    model.no_multiple_owners = Constraint(areas, owners, rule=no_multiple_owners_rule)

    # 5. Apartments by owner and area
    def apartments_by_owner_area_rule(model, j, h):
        return model.Z[j, h] == sum(model.Y[i, j, h] for i in sectors)
    model.apartments_by_owner_area = Constraint(areas, owners, rule=apartments_by_owner_area_rule)

    # 6. Minimal percentage of a sector in total program
    def minimal_sector_percentage_rule(model, i):
        return sum(model.Y[i, j, h] for j in areas for h in owners) >= beta[i] * sum(model.Y[l, j, h] for l in sectors for j in areas for h in owners)
    model.minimal_sector_percentage = Constraint(sectors, rule=minimal_sector_percentage_rule)

    # 7. Minimal average floor area of apartments in a sector
    def minimal_avg_area_rule(model, i):
        if delta[i] > 0:
            return sum(j * model.Y[i, j, h] for j in areas for h in owners) >= delta[i] * sum(model.Y[i, j, h] for j in areas for h in owners)
        else:
            return Constraint.Skip
    model.minimal_avg_area = Constraint(sectors, rule=minimal_avg_area_rule)

    # 8. Minimal floor area for a sector and owner
    def min_floor_area_rule(model, i, j, h):
        if (i, h) in alpha:
            return model.Y[i, j, h] == 0 if j < alpha[(i, h)] else Constraint.Skip
        else:
            return Constraint.Skip
    model.min_floor_area = Constraint(sectors, areas, owners, rule=min_floor_area_rule)

    # 9. No free sector apartments for housing corporation
    def no_free_apartments_corporation_rule(model, j):
        return model.Y['free', j, 'corporation'] == 0
    model.no_free_apartments_corporation = Constraint(areas, rule=no_free_apartments_corporation_rule)

    # 10. Sector ownership requirement (investor must own at least 70%)
    def sector_ownership_requirement_rule(model, h):
        if gamma[h] > 0:
            return sum(model.Y[i, j, h] for i in sectors for j in areas) >= gamma[h] * sum(model.Y[i, j, p] for i in sectors for j in areas for p in owners)
        else:
            return Constraint.Skip
    model.sector_ownership_requirement = Constraint(owners, rule=sector_ownership_requirement_rule)

    solver = SolverFactory('gurobi')
    results = solver.solve(model)

    total_profit = model.obj()
    return total_profit, results




In [ ]:
# solve for each building height
for height, profit_df in profits.items():
    print(f"Solving for height {height} floors")
    profit, _ = create_model(height, profit_df, apartments)
    print(f"Total profit for {height} floors: {profit}")

Solving for height 23 floors
Total profit for 23 floors: 17483910.863021515
Solving for height 40 floors
Total profit for 40 floors: 29622229.854210615
Solving for height 56 floors
Total profit for 56 floors: 39187429.27353782
